# Pandas

In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

# Indexación Jerárquica

Al colocar en la opción index un array multidimensional se genera una jerarquia de indices donde el arreglo correspondiente a la primera entrada es la primera capa de la jerarquia. 

In [4]:
data = Series(np.random.rand(10), 
                 index =[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2,3, 1, 2,3, 1, 2,2, 3]])
data

a  1    0.997133
   2    0.329860
   3    0.537674
b  1    0.910829
   2    0.446875
   3    0.379329
c  1    0.968902
   2    0.771363
d  2    0.265769
   3    0.518155
dtype: float64

Nota que en el arreglo anterior, las etiquetas de la primera capa (a, b,c,d) etiquetan bloques enteros. Cada bloque tiene sus propios indices que corresponden a las entradas de la segunda capa.


Al colocar un array multidimensional como indice pandas crea un arreglo de tuplas, la entrada i-esima de la j-esima tupla corresponde al i-esimo indice de la j-esima capa. Para acceder al indice jerarquico sigue funcionando el metodo index

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

 Se puede acceder a las capas del multiindice usando 'levels' 

In [96]:
capa1,capa2=data.index.levels
capa1

Index(['a', 'b', 'c', 'd'], dtype='object')

Para acceder a los bloques etiquetados por la primera capa de indices se usa la misma notación que para series o arreglos de numpy

In [6]:
data['b':'c']

b  1    0.910829
   2    0.446875
   3    0.379329
c  1    0.968902
   2    0.771363
dtype: float64

Para acceder a bloques predeterminados se usa la misma notación que para acceder a las columnas de un dataframe

In [7]:
data.loc[['b', 'd']]

b  1    0.910829
   2    0.446875
   3    0.379329
d  2    0.265769
   3    0.518155
dtype: float64

Nota que una Serie con indices jerarquicos de orden 2 es equivalente a un dataframe.


In [8]:
data[:, 2]

a    0.329860
b    0.446875
c    0.771363
d    0.265769
dtype: float64

## Una serie con índices jerárquicos puede convertirse en dataframe usando el metodo unstack 

In [9]:
# arrange it into a DataFrame

data.unstack()

,1,2,3
a,0.997133,0.329860,0.537674
b,0.910829,0.446875,0.379329
c,0.968902,0.771363,NaN
d,NaN,0.265769,0.518155


## Y deshacer tal acción con stack

In [12]:
data.unstack().stack()

a  1    0.237395
   2    0.129876
   3    0.435973
b  1    0.403339
   2    0.628142
   3    0.261277
c  1    0.218988
   2    0.229456
d  2    0.163585
   3    0.280751
dtype: float64

### Tanto las filas como las columnas pueden tener indices jerárquicos

In [17]:
frame = DataFrame(np.arange(12).reshape((4,3)),
                     index = [['a', 'a', 'b', 'b'], [1, 2, 'I', 'II']],
                     columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])


In [18]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    I        6   7        8
     II       9  10       11

In [19]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    I         6    7
     II        9   10

# Dada una serie puede asociarsele un índice jerárquico con MultiIndex

## A partir de arrays

In [21]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                             names =['state', 'color'])


MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## y a partir de tuplas 

In [24]:
multiindex = pd.MultiIndex.from_tuples([("A", 1), ("A", 2), ("A", 3), ("B", 2), ("B", 3)], names=["letters", "numbers"])
multiindex

MultiIndex([('A', 1),
            ('A', 2),
            ('A', 3),
            ('B', 2),
            ('B', 3)],
           names=['letters', 'numbers'])

## y asociar ese multindice a una serie

In [27]:
muestra = pd.Series(list(range(100, 105)),index=multiindex)
muestra

letters  numbers
A        1          100
         2          101
         3          102
B        2          103
         3          104
dtype: int64

In [28]:
muestra[("B", 2)]

103

# Groupby es útil  para agrupar datos de acuerdo a algún criterio


### Por ejemplo, al agrupar los elementos minimos con en los bloques etiquetados con "letters" o numbers   se almacenan en una nueva serie 

In [33]:
muestra.groupby(level="letters").min()

letters
A    100
B    103
dtype: int64

In [32]:
muestra.groupby(level="numbers").min()

numbers
1    100
2    101
3    102
dtype: int64

### Al aplicar "groupby" se realizan combinadamente las siguientes acciones:  expandir un objeto y aplicar una función.
### Puede usarse para agrupar conjuntos de datos grandes y operarlos

In [63]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
          ['Captive', 'Wild', 'Captive', 'Wild']]
index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]},index=index)
df

Max Speed
Animal Type              
Falcon Captive      390.0
       Wild         350.0
Parrot Captive       30.0
       Wild          20.0

In [41]:
df.groupby(level="Type").mean()

,Max Speed
Type,
Captive,210.0
Wild,185.0


In [39]:
df.groupby(level="Animal").mean()

,Max Speed
Animal,
Falcon,370.0
Parrot,25.0


# Acomodar por Capas

### swaplevel

In [27]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### sortlevel

In [35]:
frame.sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [29]:
frame.swaplevel(0,1).sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

## Summary Statistics by Level

### level
you can specify the level you want to (sum) by on a particular axis

In [36]:
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [41]:
frame.sum(level = 'color', axis= 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Manipular columnas

In [44]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one','one', 'two', 'two', 'two', 'two'],
                      'd': [0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### set_index

In [46]:
frame2 =  frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [47]:
# but still leave the colums

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

### reset_index

In [48]:
# and the opposite, move an index to the colums

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
